# Radial Average
## Importing libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

## Computing

In [2]:
error_2 = {}
DA_2 = {}
count_matrix_2 = {}
average_matrix_2 = {}
DA_5 = {}
error_5 = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]

# Extracting the radiuses with theta1 = theta2 = 0.0

engine = sx.radial_scanner(
    alpha_values, 
    np.zeros(alpha_values.shape),
    np.zeros(alpha_values.shape),
    d_r,
    starting_step=starting_step
)
engine.scan(turn_sampling[0], turn_sampling[-1], batch_size=batch_size)
all_radiuses = engine.extract_DA(turn_sampling)


Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: nan
Elapsed time for whole iteration: 2.43
Time per single iteration: 2.368154293153344e-05
r: 20.0 . Turns to do: 50 . Min found: 50
Active radiuses: 1025 / 1025
Sample size per active radius: 2
Expected execution time for step: 2.43
Elapsed time for whole iteration: 2.39
Time per single iteration: 2.332088423938286e-05
r: 30.0 . Turns to do: 50 . Min found: 1
Active radiuses: 693 / 1025
Sample size per active radius: 3
Expected execution time for step: 2.42
Elapsed time for whole iteration: 2.26
Time per single iteration: 2.176203367628691e-05
r: 45.0 . Turns to do: 5 . Min found: 0
TOTAL ELAPSED TIME IN SECONDS: 7.08


### 2D simple integral

In [3]:
skips = [1]
while True:
    if (samples - 1) // skips[-1] > 4:
        skips.append(skips[-1] * 2)
    else:
        break

for i in skips:
    alpha = alpha_preliminary_values[::i]
    cutted_radiuses = all_radiuses[::i]
    value = integrate.simps(cutted_radiuses ** 2, alpha, axis=0)
    less_value = integrate.simps(cutted_radiuses[::2] ** 2, alpha[::2], axis=0)
    uncertainty = np.abs((value - less_value))

    DA = np.sqrt(value / 2)
    uncertainty = 0.5 * np.power(value / 2, -0.5) * uncertainty
    DA_5[cutted_radiuses.shape] = np.asarray(DA)
    error_5[cutted_radiuses.shape] = uncertainty 

### Actual angular averaging

In [10]:
values = []
for i in tqdm(range(len(turn_sampling))):
    radiuses = all_radiuses[:, i]

    r, a, th1, th2 = sx.full_track_particles(
        radiuses,
        alpha_values,
        np.zeros(alpha_values.shape),
        np.zeros(alpha_values.shape),
        turn_sampling[i])

    count_matrix, avg_matrix, temp_values = sx.accumulate_and_return(r, a, th1, th2, n_subdivisions)
    
    count_total, matrix_total, result_total = sx.recursive_accumulation(count_matrix, avg_matrix)
    
    values.append(result_total)
    
    count_matrix_2[(samples, turn_sampling[i])] = count_total
    average_matrix_2[(samples, turn_sampling[i])] = matrix_total

steps = [1]
while True:
    if (len(values[0][0]) - 1) / steps[-1] > 4:
        steps.append(steps[-1] * 2)
    else:
        break

for jump in steps:
    for j in range(len(values[0])):
        DA = []
        error = []
        for i in range(len(turn_sampling)):
            DA.append(np.power(integrate.simps(values[i][j][::jump], dx = d_preliminar_alpha * jump) * 0.5, 1/4))
            temp = np.power(integrate.simps(values[i][j][::jump * 2], dx = d_preliminar_alpha * jump * 2) * 0.5, 1/4)
            error.append(np.absolute(DA[-1] - temp))
        DA_2[(len(values[i][j][::jump]), 2 ** j)] = DA
        error_2[(len(values[i][j][::jump]), 2 ** j)] = error

100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


## Saving data

In [12]:
with open(savepath + "data/DA_2.pkl", 'wb') as f:
    pickle.dump(DA_2, f, protocol=4)
    
with open(savepath + "data/error_2.pkl", 'wb') as f:
    pickle.dump(error_2, f, protocol=4)
    
with open(savepath + "data/count_matrix_2.pkl", 'wb') as f:
    pickle.dump(count_matrix_2, f, protocol=4)
    
with open(savepath + "data/avg_matrix_2.pkl", 'wb') as f:
    pickle.dump(average_matrix_2, f, protocol=4)
    
with open(savepath + "data/DA_5.pkl", 'wb') as f:
    pickle.dump(DA_5, f, protocol=4)
    
with open(savepath + "data/error_5.pkl", 'wb') as f:
    pickle.dump(error_5, f, protocol=4)